In [3]:
from sentinelhub import SHConfig

config = SHConfig()
config.sh_client_id = "7a3098a7-bfcd-4bff-baa4-6104fef14169"
config.sh_client_secret = "bTOS03EAVbfImGlrGqHjMkstr9IJ0aQ1"
config.save()

from sentinelhub import SentinelHubRequest, MimeType, CRS, BBox, bbox_to_dimensions, DataCollection

# Define latitude & longitude
latitude = 28.108601   # Change to your location
longitude = 96.509742

# Define bounding box (small area)
bbox = BBox([longitude - 0.01, latitude - 0.01, longitude + 0.01, latitude + 0.01], CRS.WGS84)

# Define image resolution
resolution = 2   # 10m per pixel

# Get dimensions
size = bbox_to_dimensions(bbox, resolution=resolution)

# Create request for Sentinel-2 image
request = SentinelHubRequest(
    evalscript="""
    // Extract RGB image
    function setup() {
        return {
            input: ["B04", "B03", "B02"],
            output: { bands: 3 }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
    """,
    input_data=[SentinelHubRequest.input_data(DataCollection.SENTINEL2_L2A, maxcc=0.1, time_interval=("2024-01-01", "2024-12-31"))],  # Fixed maxcc
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=bbox,
    size=size
)

# Get image
image = request.get_data()[0]

# Save image
import imageio
imageio.imwrite("/home/arshlaan/projext/codathon/satellite_image.png", image)

print("Image downloaded as satellite_image.png")

import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("/home/arshlaan/projext/codathon/wildfire.h5")

def preprocess_image(img_path):
    """
    Preprocess a single image to match the model's input format.

    Args:
        img_path (str): Path to the image file.

    Returns:
        np.array: Preprocessed image ready for model input.
    """
    image = cv2.imread(img_path)  # Read the image
    image = cv2.resize(image, (32, 32))  # Resize to match training dimensions
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0,1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return np.array(image)

def predict_image(img_path, class_names):
    """
    Predict the class of an image using the trained model.

    Args:
        img_path (str): Path to the image file.
        class_names (list): List of class labels.

    Returns:
        str: Predicted class with confidence percentage.
    """
    image = preprocess_image(img_path)
    predictions = model.predict(image)  # Get model predictions
    predicted_class = np.argmax(predictions)  # Get index of highest probability
    confidence = np.max(predictions) * 100  # Convert to percentage

    return f"Predicted: {class_names[predicted_class]} ({confidence:.2f}% confidence)"

# Define class names based on your dataset
class_names = ["nofire", "fire"]  # Replace with actual class names

# Example usage
img_path = "//home/arshlaan/projext/codathon/satellite_image.png"  # Provide the image path
result = predict_image(img_path, class_names)
print(result)

Image downloaded as satellite_image.png


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
Predicted: nofire (99.75% confidence)
